# Face Recognition Project
- **Process Steps**
    
     - **Loading Data** we started making fetch for all directories we have in our dataset and star to make it our shape which is triple shape $( Anchor , Positive , Negative )$ .
           
         - **Where :**
             - **Anchor** is the image for some one in dataset.
             - **Positive** is an image of the same person how is in **Anchor**.
             - **Negative** is an image for anthor one .
               
     -**Creating Data Pipeline**   we have used tensorflow pipline for in creasing performance of loading and make some operations like caching for redusing time make patch which is good for learning model
     
     -**Creating Model**
         - **1** we have made our model depending on MobilenetV3
         - **2** we added sum layer on top of model
     -**Evaluating Result**
         we have two man validation metrics .
         - **VAL** which show persent of getting good responce between same person $TP/(TP+FP)$
         - **FAR** which show the presentage of in correct decitions of diferent persons $ FN/(FN+TN) $
     
     -**Making Security For Verification**
     
     ---------------------------------------------------- **End** ---------------------------------------------------------

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_hub as hub
import os
import io
import seaborn as sns
from keras.layers import Dense,Dropout,Input,BatchNormalization,Lambda
from keras.models import Model
from keras.optimizers import Adam
import argparse
from sklearn.metrics import confusion_matrix , accuracy_score
from tensorflow.keras import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation,RandomFlip,RandomCrop,PreprocessingLayer
#import tensorflow_model_optimization as tfmot

from google.colab import drive
drive.mount("drive")
%matplotlib inline

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


# dataset

## loading data

In [ ]:
# BaseDatasetPath="./Dataset/Extracted Faces/Extracted Faces/"
# TrainUsersDatasetPath="./Dataset/Extracted Faces/team/"
# TestUsersDatasetPath="./Dataset/Extracted Faces/test/"
BaseDatasetPath="./drive/MyDrive/Extracted Faces/Extracted Faces/"
TrainUsersDatasetPath="./drive/MyDrive/Extracted Faces/train/"
TestUsersDatasetPath="./drive/MyDrive/Extracted Faces/test/"
def DatasetPaths(BaseDatapath,UserDatapath):
    Base=os.listdir(BaseDatapath)
    User=os.listdir(UserDatapath)
    Basefiles=[]
    UserFiles=[]
    UserLabels=[]
    for folder in Base:
        for file in os.listdir(BaseDatapath+folder):
            Basefiles.append(BaseDatapath+folder+"/"+file)
    for folder in User:
        for file in os.listdir(UserDatapath+folder):
            UserFiles.append(file)
            UserLabels.append(folder+'/')

    return Basefiles,np.array(UserFiles),np.array(UserLabels)

In [ ]:
Basefiles,UserFiles,UserLabels=DatasetPaths(BaseDatasetPath,TrainUsersDatasetPath)

TrainClasses=np.unique(UserLabels)
TrainClassesCount=len(TrainClasses)

_,TestUserFiles,TestUserLabels=DatasetPaths(BaseDatasetPath,TestUsersDatasetPath)
TestClasses=np.unique(TestUserLabels)
TestClassesCount=len(TestClasses)

## data augmetation

In [ ]:
mask=np.zeros(shape=(224,224,3))
mask[:,:,0]=200
mask[:,:,1]=100
mask[:,:,2]=200
mask=tf.cast(mask/255,tf.float32)
FliPer=RandomFlip(mode="horizontal",)
Rotater=RandomRotation([-0.135,0.135])
def PreProcessInput(Image,num):
    if num ==0:
        Image=FliPer(Image)
    elif num==1:
        Image= 0.75*Image+0.25*mask
    if num<=2:
        return Rotater(Image)

    else:
        return Image

## Loading Images Function

In [ ]:
@tf.function
def load_image(Anchor,Positive,Nagative,State):

    Anchor=tf.io.read_file(Anchor)
    Anchor=tf.image.decode_jpeg(Anchor)
    Anchor = tf.cast(Anchor, tf.float32)
    Anchor = tf.image.resize(Anchor, [224,224], method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    ranA=tf.random.uniform(shape=[1],minval=0,maxval=6,dtype=tf.int32)

    Positive=tf.io.read_file(Positive)
    Positive=tf.image.decode_jpeg(Positive)
    Positive = tf.cast(Positive, tf.float32)
    Positive = tf.image.resize(Positive, [224,224], method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    ranB=tf.random.uniform(shape=[1],minval=0,maxval=6,dtype=tf.int32)

    Negative=tf.io.read_file(Nagative)
    Negative=tf.image.decode_jpeg(Negative)
    Negative = tf.cast(Negative, tf.float32)
    Negative = tf.image.resize(Negative, [224,224], method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    ranN=tf.random.uniform(shape=[1],minval=0,maxval=6,dtype=tf.int32)
    if State:
        Anchor=PreProcessInput(Anchor/255,ranA)
        Positive=PreProcessInput(Positive/255,ranB)
        Negative=PreProcessInput(Negative/255,ranN)
    else:
        Anchor=Anchor/255
        Positive=Positive/255
        Negative=Negative/255

    return (Anchor,Positive,Negative)

In [ ]:
def DatasetTripletsGenerator(State):
    if State:
        ImagesName=UserFiles
        ImagesLabel=UserLabels
        ClassesCount=TrainClassesCount
        Classes=TrainClasses
        UserPath=TrainUsersDatasetPath
    else:
        ImagesName=TestUserFiles
        ImagesLabel=TestUserLabels
        ClassesCount=TestClassesCount
        Classes=TestClasses
        UserPath=TestUsersDatasetPath


    for i in range(ClassesCount):
        class_=Classes[i]
        files=ImagesName[ImagesLabel==class_]
        files_num=len(files)
        for index in range(files_num-1):
            for j in range(index+1,files_num):
                ancore=UserPath+class_+files[index]
                positive=UserPath+class_+files[j]
                neg_folder=class_
                while neg_folder== class_:
                    neg_folder=np.random.choice(Classes)
                negative=UserPath+neg_folder+np.random.choice(ImagesName[ImagesLabel==neg_folder])
                negative1=np.random.choice(Basefiles)
                if np.random.randint(0,high=2)==1:
                    negative=negative1
                yield ancore,positive,negative,State

In [ ]:
TrainData=tf.data.Dataset.from_generator(DatasetTripletsGenerator,args=[True],output_types=(tf.string,tf.string,tf.string,tf.bool),output_shapes=((),(),(),()),name="DataLoaderPipeline")
TrainData=TrainData.map(load_image)
TrainData=TrainData.batch(20).shuffle(buffer_size=100)
# TrainData=TrainData.prefetch(buffer_size=1)
# data=data.cache()
TestData=tf.data.Dataset.from_generator(DatasetTripletsGenerator,args=[False],output_types=(tf.string,tf.string,tf.string,tf.bool),output_shapes=((),(),(),()),name="DataLoaderPipeline")
TestData=TestData.map(load_image).batch(20)

In [ ]:
for ancore,positive,negative in TrainData.take(1):
    fig,(ax1,ax2,ax3)=plt.subplots(1,3,sharex=True,sharey=True)
    ax1.imshow(ancore[0])
    ax2.imshow(positive[0])
    ax3.imshow(negative[0])

# Model

In [ ]:
class DistanceLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    def call(self,anchor,positive,negative):
        dis_ap=tf.reduce_sum(tf.square(anchor - positive), 1)  ## distance between anchor and positive
        dis_an=tf.reduce_sum(tf.square(anchor - negative), 1)   ## distance between anchor and negative
        return  dis_ap , dis_an

In [ ]:
def GetEncoder():
#     /drive/MyDrive/Model/
    return tf.keras.models.load_model("./drive/MyDrive/liteFaceRecognitionModel/FaceModelV5")
def SiameseNetwork(inputshape=(224,224,3)):
    An_input=Input(shape=inputshape)

    Po_input=Input(shape=inputshape)

    Ne_input=Input(shape=inputshape)

    encoder=GetEncoder()

    An_embeding=encoder(An_input)
    Po_embeding=encoder(Po_input)
    Ne_embeding=encoder(Ne_input)


    distanc=DistanceLayer()(An_embeding,Po_embeding,Ne_embeding) #return distance between (A and B) and (A and N)

    return Model(inputs=[An_input,Po_input,Ne_input],outputs=distanc)

In [ ]:
siames_net=SiameseNetwork()

In [ ]:
siames_net.layers[3].layers[0].trainable=False

In [ ]:
class SiamesModel(Model):
    def __init__(self,siames_net,DesiredDistance):
        super(SiamesModel, self).__init__()

        self.Model=siames_net
        self.DesiredDistance=DesiredDistance
        self.LossTracker=tf.keras.metrics.Mean(name="Loss")

        self.VALTracker=tf.keras.metrics.Mean(name="VAL")

        self.PmeanTracker=tf.keras.metrics.Mean(name="P_mean")

        self.PmaxTracker=tf.keras.metrics.Mean(name="P_max")

        self.PstdTracker=tf.keras.metrics.Mean(name="P_std")

        self.FARTracker=tf.keras.metrics.Mean(name="FAR")

        self.N_meanTracker=tf.keras.metrics.Mean(name="N_mean")

        self.NstdTracker=tf.keras.metrics.Mean(name="N_std")
        self.NminTracker=tf.keras.metrics.Mean(name="N_min")

    def call(self,data):
        return self.Model(data)

    def train_step(self,data):
        with tf.GradientTape() as Tape:
            AP_distanc,AN_distance=self.Model(data)
            loss=self.TripLoss(AP_distanc,AN_distance)
            gradients=Tape.gradient(loss,self.Model.trainable_weights)
            self.optimizer.apply_gradients(zip(gradients, self.Model.trainable_weights))
        self.DistanceEval(AP_distanc,AN_distance)
        self.LossTracker.update_state(loss)
        return {"VAL":self.VALTracker.result(),
                "P_mean":self.PmeanTracker.result(),
                "P_max":self.PmaxTracker.result(),
                "P_std":self.PstdTracker.result(),
                "FAR":self.FARTracker.result(),
                "N_mean":self.N_meanTracker.result(),
                "N_min":self.NminTracker.result(),
                "N_std":self.NstdTracker.result(),
                "Loss":self.LossTracker.result()}


    def test_step(self, data):
        AP_distanc,AN_distance=self.Model(data)
        loss=self.TripLoss(AP_distanc,AN_distance)
        self.LossTracker.update_state(loss)
        self.DistanceEval(AP_distanc,AN_distance)
        return {"VAL":self.VALTracker.result(),
                "P_mean":self.PmeanTracker.result(),
                "P_max":self.PmaxTracker.result(),
                "P_std":self.PstdTracker.result(),
                "FAR":self.FARTracker.result(),
                "N_mean":self.N_meanTracker.result(),
                "N_min":self.NminTracker.result(),
                "N_std":self.NstdTracker.result(),
                "Loss":self.LossTracker.result()}



    def TripLoss(self,ap_distance,an_distance):
        return tf.reduce_mean(tf.maximum(ap_distance-0.2*self.DesiredDistance,0)+tf.maximum(self.DesiredDistance-an_distance, 0.0))


    @property
    def metrics(self):
        return [self.LossTracker,self.VALTracker,self.PmaxTracker,self.PmeanTracker,self.PstdTracker,self.FARTracker,self.N_meanTracker,self.NminTracker,self.NstdTracker]

    def DistanceEval(self,P_distance,N_distance):

        P_pred,N_pred=self.TDEvaluation(P_distance,N_distance)
        PCDCount=tf.size(tf.where(P_pred))

        VAL=PCDCount/tf.size(P_pred)
        self.VALTracker.update_state(VAL)

        NCDcount=tf.size(tf.where(N_pred))
        FAR=1-(NCDcount/tf.size(P_pred))
        self.FARTracker.update_state(FAR)
        P_mean=tf.reduce_mean(P_distance)
        self.PmeanTracker.update_state(P_mean)
        N_mean=tf.reduce_mean(N_distance)
        self.N_meanTracker.update_state(N_mean)
        P_std=tf.math.reduce_std(P_distance)
        self.PstdTracker.update_state(P_std)
        N_std=tf.math.reduce_std(N_distance)
        self.NstdTracker.update_state(N_std)
        P_max=tf.reduce_max(P_distance)
        self.PmaxTracker.update_state(P_max)
        N_min=tf.reduce_min(N_distance)
        self.NminTracker.update_state(N_min)

    def TDEvaluation(self,P_distance,N_distance):
        return tf.cast(P_distance<=self.DesiredDistance,dtype=tf.int8),tf.cast(N_distance>self.DesiredDistance,dtype=tf.int8)

In [ ]:
DesiredDistance=1
Optimizer= Adam(learning_rate=1e-4)
Siamesmodel=SiamesModel(siames_net,DesiredDistance)
Siamesmodel.compile(optimizer=Adam(1e-4),weighted_metrics=[])

In [ ]:
Siamesmodel.fit(TrainData,validation_data=TestData,epochs=2)

Epoch 1/2
43/43 [==============================] - 139s 2s/step - VAL: 1.0000 - P_mean: 0.0237 - P_max: 0.0909 - P_std: 0.0222 - FAR: 0.0000e+00 - N_mean: 1.8952 - N_min: 1.5068 - N_std: 0.1509 - Loss: 2.3350e-04 - val_Loss: 0.0026 - val_VAL: 1.0000 - val_P_max: 0.1197 - val_P_mean: 0.0302 - val_P_std: 0.0316 - val_FAR: 0.0250 - val_N_mean: 1.8053 - val_N_min: 1.1383 - val_N_std: 0.2300
Epoch 2/2
43/43 [==============================] - 136s 2s/step - VAL: 1.0000 - P_mean: 0.0192 - P_max: 0.0774 - P_std: 0.0190 - FAR: 0.0012 - N_mean: 1.9005 - N_min: 1.5267 - N_std: 0.1430 - Loss: 5.1480e-04 - val_Loss: 0.0028 - val_VAL: 1.0000 - val_P_max: 0.1111 - val_P_mean: 0.0292 - val_P_std: 0.0290 - val_FAR: 0.0500 - val_N_mean: 1.8685 - val_N_min: 1.3016 - val_N_std: 0.2173


# Converting to TFlite

In [ ]:
siames_net.layers[3].save("./drive/MyDrive/liteFaceRecognitionModel/FaceModelV5")

In [ ]:

converter.optimizations=[tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types=[tf.float16]
tflitemodel=converter.convert()

In [ ]:
with open("./drive/MyDrive/liteFaceRecognitionModel/FinalFaceModel30Epoce.tflite","wb") as file:
    file.write(tflitemodel)